In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = "sk-e7f0cd5b1a7a4d9f93e29f00a8145705"
print(api_key)

sk-e7f0cd5b1a7a4d9f93e29f00a8145705


In [3]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs_2.4.x_en/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [4]:
len(text_lines)

72

In [5]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [6]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs_2.4.x_en/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [7]:
len(text_lines)

72

In [8]:
print(text_lines[1])

Operational FAQ

<!-- TOC -->


<!-- /TOC -->

###


In [9]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [10]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

In [11]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [12]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.02752976  0.0608853   0.00388525 -0.00215193 -0.02774976 -0.0118618
 -0.04020916 -0.06023417 -0.03813156  0.0100272 ]


In [13]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [14]:
import faiss
import numpy as np

class FaissVectorStore:
    def __init__(self, dim):
        self.dim = dim  # 向量维度
        self.index = faiss.IndexFlatL2(dim)  # FAISS 索引
        self.vectors = []  # 存储原始向量
        self.texts = []    # 存储向量对应文本
        self.collection_exists = False  # 标记集合是否存在（核心：替代 Milvus 的集合存在判断）

    def has_collection(self, collection_name):
        """判断集合是否存在（对应 Milvus 的 has_collection）"""
        # 这里靠 self.collection_exists 标记，实际项目可按需求扩展（如多集合管理）
        print(f"🔍 检查集合 {collection_name} 是否存在：{'是' if self.collection_exists else '否'}")
        return self.collection_exists

    def drop_collection(self, collection_name):
        """删除集合（对应 Milvus 的 drop_collection）"""
        if self.collection_exists:
            # 清空索引和数据，重置集合状态
            self.index = faiss.IndexFlatL2(self.dim)
            self.vectors = []
            self.texts = []
            self.collection_exists = False
            print(f"🗑️  集合 {collection_name} 已删除")
        else:
            print(f"⚠️  集合 {collection_name} 不存在，无需删除")

    def create_collection(self, collection_name):
        """创建集合（对应 Milvus 的 create_collection）"""
        # 先判断集合是否存在，存在则删除（对应你原本的逻辑）
        if self.has_collection(collection_name):
            self.drop_collection(collection_name)
        # 标记集合为存在状态
        self.collection_exists = True
        print(f"✅ 集合 {collection_name} 创建成功")
        return True

    def insert(self, vectors, texts):
        """插入向量（对应 Milvus 的 insert）"""
        if not self.collection_exists:
            raise ValueError("❌ 请先创建集合再插入数据！")
        vectors = np.array(vectors, dtype=np.float32)
        self.index.add(vectors)
        self.vectors.extend(vectors)
        self.texts.extend(texts)
        print(f"✅ 向集合插入 {len(vectors)} 条数据成功")
        return len(vectors)

    def search(self, query_vector, top_k=3):
        """检索相似向量（对应 Milvus 的 search）"""
        if not self.collection_exists:
            raise ValueError("❌ 请先创建集合再检索数据！")
        query_vector = np.array([query_vector], dtype=np.float32)
        distances, indices = self.index.search(query_vector, top_k)
        similar_texts = [self.texts[i] for i in indices[0] if i < len(self.texts)]
        return similar_texts

# ------------------- 测试：完全复现你原本的 Milvus 逻辑 -------------------
if __name__ == "__main__":
    # 1. 初始化向量存储（假设文本嵌入维度为 3，实际用 768）
    vector_store = FaissVectorStore(dim=3)
    collection_name = "my_rag_collection"

    # 2. 核心逻辑：判断集合是否存在 → 存在则删除 → 创建新集合（和你原本代码一致）
    if vector_store.has_collection(collection_name):
        vector_store.drop_collection(collection_name)
    vector_store.create_collection(collection_name)

    # 3. 插入数据（模拟文本嵌入后的向量）
    test_vectors = [
        [0.1, 0.2, 0.3],
        [0.4, 0.5, 0.6],
        [0.7, 0.8, 0.9]
    ]
    test_texts = [
        "文本1：机器学习基础",
        "文本2：深度学习实战",
        "文本3：向量数据库原理"
    ]
    vector_store.insert(test_vectors, test_texts)

    # 4. 检索相似文本（模拟 RAG 查询）
    query_vector = [0.15, 0.25, 0.35]  # 模拟问题嵌入向量
    similar_texts = vector_store.search(query_vector, top_k=2)
    print("\n📊 检索到的相似文本：")
    for i, text in enumerate(similar_texts, 1):
        print(f"{i}. {text}")

🔍 检查集合 my_rag_collection 是否存在：否
🔍 检查集合 my_rag_collection 是否存在：否
✅ 集合 my_rag_collection 创建成功
✅ 向集合插入 3 条数据成功

📊 检索到的相似文本：
1. 文本1：机器学习基础
2. 文本2：深度学习实战


In [15]:
import faiss
import numpy as np

class FaissVectorStore:
    def __init__(self):
        self.index = None  # 初始化索引为None（未创建集合时无索引）
        self.vectors = []  # 存储原始向量
        self.texts = []    # 存储向量对应文本
        self.collection_exists = False  # 标记集合是否存在

    def has_collection(self, collection_name):
        print(f"🔍 检查集合 {collection_name} 是否存在：{'是' if self.collection_exists else '否'}")
        return self.collection_exists

    def drop_collection(self, collection_name):
        if self.collection_exists:
            self.index = None  # 清空索引
            self.vectors = []
            self.texts = []
            self.collection_exists = False
            print(f"🗑️  集合 {collection_name} 已删除")
        else:
            print(f"⚠️  集合 {collection_name} 不存在，无需删除")

    # ------------------- 核心修改：适配 Faiss 的 create_collection -------------------
    def create_collection(
        self,
        collection_name: str,
        dimension: int,  # 对应 Milvus 的 dimension（向量维度）
        metric_type: str = "IP",  # 对应 Milvus 的 metric_type（距离计算方式）
        consistency_level: str = "Strong"  # Faiss 无需此配置，仅保留参数兼容
    ):
        # 1. 先删除已存在的集合（保持和 Milvus 逻辑一致）
        if self.has_collection(collection_name):
            self.drop_collection(collection_name)
        
        # 2. 根据 metric_type 初始化 Faiss 索引（核心：Faiss 用不同索引类对应距离类型）
        if metric_type == "IP":
            # IP（内积）：Faiss 用 IndexFlatIP 实现内积距离计算
            self.index = faiss.IndexFlatIP(dimension)
        elif metric_type == "L2":
            # L2（欧氏距离）：Faiss 用 IndexFlatL2 实现，兼容 Milvus 默认逻辑
            self.index = faiss.IndexFlatL2(dimension)
        else:
            raise ValueError(f"❌ Faiss 暂不支持该 metric_type：{metric_type}，仅支持 'IP' 或 'L2'")
        
        # 3. 标记集合创建成功（Faiss 无分布式特性，consistency_level 仅注释说明，无需实际配置）
        self.collection_exists = True
        print(f"✅ 集合 {collection_name} 创建成功")
        print(f"  - 向量维度：{dimension}")
        print(f"  - 距离类型：{metric_type}（内积）")
        print(f"  - 说明：Faiss 本地索引无需 'consistency_level' 配置，{consistency_level} 仅作兼容保留")

    def insert(self, vectors, texts):
        if not self.collection_exists or self.index is None:
            raise ValueError("❌ 请先调用 create_collection 创建集合再插入数据！")
        # Faiss 要求向量为 float32 类型，需转换格式
        vectors = np.array(vectors, dtype=np.float32)
        self.index.add(vectors)  # 插入向量到 Faiss 索引
        self.vectors.extend(vectors)
        self.texts.extend(texts)
        print(f"✅ 向集合 {collection_name} 插入 {len(vectors)} 条数据成功")

    def search(self, query_vector, top_k=3):
        if not self.collection_exists or self.index is None:
            raise ValueError("❌ 请先调用 create_collection 创建集合再检索数据！")
        query_vector = np.array([query_vector], dtype=np.float32)
        distances, indices = self.index.search(query_vector, top_k)  # Faiss 检索
        # 提取检索结果对应的文本
        similar_texts = [self.texts[i] for i in indices[0] if i < len(self.texts)]
        return similar_texts


# ------------------- 测试：完全复用你原本的调用格式 -------------------
if __name__ == "__main__":
    # 1. 初始化 Faiss 向量存储
    vector_store = FaissVectorStore()
    collection_name = "my_rag_collection"
    embedding_dim = 768  # 示例：文本嵌入常用维度（如 BERT 类模型输出）

    # 2. 完全复用你原本的 Milvus create_collection 调用格式！
    vector_store.create_collection(
        collection_name=collection_name,
        dimension=embedding_dim,
        metric_type="IP",  # 内积距离，和 Milvus 配置一致
        consistency_level="Strong"  # 保留参数，Faiss 无需实际配置
    )

    # 3. 插入示例数据（模拟文本嵌入后的向量）
    test_vectors = [np.random.random(embedding_dim).astype("float32") for _ in range(5)]
    test_texts = [f"文本{i+1}：RAG 示例内容" for i in range(5)]
    vector_store.insert(test_vectors, test_texts)

    # 4. 检索示例（模拟问题嵌入向量）
    query_vector = np.random.random(embedding_dim).astype("float32")
    results = vector_store.search(query_vector, top_k=2)
    print(f"\n📊 检索到的相似文本（top 2）：")
    for idx, text in enumerate(results, 1):
        print(f"{idx}. {text}")

🔍 检查集合 my_rag_collection 是否存在：否
✅ 集合 my_rag_collection 创建成功
  - 向量维度：768
  - 距离类型：IP（内积）
  - 说明：Faiss 本地索引无需 'consistency_level' 配置，Strong 仅作兼容保留
✅ 向集合 my_rag_collection 插入 5 条数据成功

📊 检索到的相似文本（top 2）：
1. 文本3：RAG 示例内容
2. 文本4：RAG 示例内容


In [16]:
from tqdm import tqdm
import numpy as np  # 需导入numpy处理向量格式

# ------------------- 1. 生成文档嵌入（你的原有代码，无需改） -------------------
doc_embeddings = embedding_model.encode_documents(text_lines)  # 假设输出是 list 或 numpy 数组

# ------------------- 2. 数据格式转换（核心修改：适配 Faiss） -------------------
data = []
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

# 拆解为 Faiss 需要的「向量列表」和「文本列表」，并转换向量为 float32 类型
insert_vectors = []
insert_texts = []
for item in data:
    # Faiss 要求向量是 float32 类型的 numpy 数组（避免精度问题）
    vec = np.array(item["vector"], dtype=np.float32)
    insert_vectors.append(vec)
    insert_texts.append(item["text"])

# ------------------- 3. 调用 FaissVectorStore 插入数据（修改调用方式） -------------------
vector_store.insert(vectors=insert_vectors, texts=insert_texts)  # 对应封装的 insert 方法

Creating embeddings: 100%|█████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 246723.76it/s]

✅ 向集合 my_rag_collection 插入 72 条数据成功


### 构建RAG

- 确保 FaissVectorStore 类定义正确

In [17]:
import faiss
import numpy as np

class FaissVectorStore:
    def __init__(self):
        self.index = None
        self.vectors = []
        self.texts = []
        self.collection_exists = False

    def has_collection(self, collection_name):
        return self.collection_exists

    def drop_collection(self, collection_name):
        if self.collection_exists:
            self.index = None
            self.vectors = []
            self.texts = []
            self.collection_exists = False
            print(f"🗑️  集合 {collection_name} 已删除")

    def create_collection(
        self,
        collection_name: str,
        dimension: int,
        metric_type: str = "IP",
        consistency_level: str = "Strong"
    ):
        if self.has_collection(collection_name):
            self.drop_collection(collection_name)
        
        if metric_type == "IP":
            self.index = faiss.IndexFlatIP(dimension)
        elif metric_type == "L2":
            self.index = faiss.IndexFlatL2(dimension)
        else:
            raise ValueError(f"❌ Faiss 仅支持 'IP'/'L2'，不支持 {metric_type}")
        
        self.collection_exists = True
        print(f"✅ 集合 {collection_name} 创建成功（维度：{dimension}，距离：{metric_type}）")

    def insert(self, vectors, texts):
        if not self.collection_exists or self.index is None:
            raise ValueError("❌ 请先创建集合！")
        vectors = np.array(vectors, dtype=np.float32)
        self.index.add(vectors)
        self.vectors.extend(vectors)
        self.texts.extend(texts)
        print(f"✅ 插入 {len(vectors)} 条数据成功")

    def search(self, query_vector, top_k=3):
        if not self.collection_exists or self.index is None:
            raise ValueError("❌ 请先创建集合！")
        query_vector = np.array([query_vector], dtype=np.float32)
        distances, indices = self.index.search(query_vector, top_k)
        return [self.texts[i] for i in indices[0] if i < len(self.texts)]

    def search_with_distance(self, query_vector, top_k=3):
        if not self.collection_exists or self.index is None:
            raise ValueError("❌ 请先创建集合并插入数据！")
        
        query_vector = np.array([query_vector], dtype=np.float32)
        distances, indices = self.index.search(query_vector, top_k)
        
        similar_texts = []
        valid_distances = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx < len(self.texts):
                similar_texts.append(self.texts[idx])
                valid_distances.append(float(dist))
        
        return similar_texts, valid_distances

- 按顺序执行 “初始化→创建集合→插入数据”

In [18]:
!pip install sentence-transformers

In [19]:
# ------------------- 必须先执行的步骤 -------------------
# 1. 初始化向量存储实例
vector_store = FaissVectorStore()

# 2. 定义集合名称和向量维度（维度必须与你的嵌入模型输出一致）
collection_name = "my_rag_collection"
embedding_dim = 384  # 示例：如果用 all-MiniLM-L6-v2 模型，维度是 384

# 3. 创建集合（必须执行！）
vector_store.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP"  # 内积距离，与检索逻辑一致
)

# 4. 准备你的知识库数据（替换为你的实际文本）
text_lines = [
    "判断 Milvus 是否启动成功的方法：1. 用 docker ps 查看容器状态为 Up；2. 查看日志是否有 'Milvus started successfully'；3. 客户端连接无错误。",
    "Milvus 启动失败常见原因：端口占用、内存不足、镜像版本不兼容。",
    "Milvus 是开源向量数据库，支持海量向量存储和检索。"
]

# 5. 生成文档嵌入向量（替换为你的嵌入模型）
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # 示例模型
doc_embeddings = embedding_model.encode(text_lines)  # 生成向量

# 6. 插入数据到 Faiss（必须执行！）
insert_vectors = [np.array(vec, dtype=np.float32) for vec in doc_embeddings]  # 转换格式
insert_texts = text_lines
vector_store.insert(vectors=insert_vectors, texts=insert_texts)
# -------------------------------------------------------

✅ 集合 my_rag_collection 创建成功（维度：384，距离：IP）
✅ 插入 3 条数据成功


- 执行检索代码

In [20]:
# 1. 问题定义
question = "How do I know if Milvus has started successfully?"

# 2. 生成问题嵌入向量（核心修改：将 encode_queries 改为 encode）
query_embedding = embedding_model.encode([question])[0]  # 用 encode 替代 encode_queries

# 3. Faiss检索
top_k = 3
similar_texts, distances = vector_store.search_with_distance(
    query_vector=query_embedding, 
    top_k=top_k
)

# 4. 整理并打印结果
import json
retrieved_lines_with_distances = list(zip(similar_texts, distances))
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        "判断 Milvus 是否启动成功的方法：1. 用 docker ps 查看容器状态为 Up；2. 查看日志是否有 'Milvus started successfully'；3. 客户端连接无错误。",
        0.435508131980896
    ],
    [
        "Milvus 是开源向量数据库，支持海量向量存储和检索。",
        0.2982614040374756
    ],
    [
        "Milvus 启动失败常见原因：端口占用、内存不足、镜像版本不兼容。",
        0.2323351502418518
    ]
]


### 使用 LLM 获取 RAG 响应

In [21]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [22]:
context

"判断 Milvus 是否启动成功的方法：1. 用 docker ps 查看容器状态为 Up；2. 查看日志是否有 'Milvus started successfully'；3. 客户端连接无错误。\nMilvus 是开源向量数据库，支持海量向量存储和检索。\nMilvus 启动失败常见原因：端口占用、内存不足、镜像版本不兼容。"

In [23]:
question

'How do I know if Milvus has started successfully?'

In [27]:
# 定义系统提示
SYSTEM_PROMPT = """
你是一个基于检索信息的问答助手。请严格遵循以下规则：
1. 仅使用<context>中提供的信息回答<question>中的问题，不允许使用外部知识。
2. 若上下文信息不足，直接说明"根据提供的信息，无法回答该问题"。
3. 回答需简洁准确，用英文组织语言。
4. 完成英文回答后，必须提供精准的中文翻译，放在<translated>标签内。
"""

USER_PROMPT = f"""
<context>
{context}
</context>

<question>
{question}
</question>

# 任务要求：
# 1. 首先，用英文回答<question>中的问题，回答必须严格基于<context>内的信息，禁止编造。
# 2. 如果<context>信息不足以回答问题，请直接输出："Based on the provided information, I can't answer this question."
# 3. 然后，在<translated></translated>标签内，提供上述英文回答的精准中文翻译。
# 4. 输出格式示例：
# [英文回答]
# <translated>
# [中文翻译]
# </translated>
"""

In [28]:
USER_PROMPT

'\n<context>\n判断 Milvus 是否启动成功的方法：1. 用 docker ps 查看容器状态为 Up；2. 查看日志是否有 \'Milvus started successfully\'；3. 客户端连接无错误。\nMilvus 是开源向量数据库，支持海量向量存储和检索。\nMilvus 启动失败常见原因：端口占用、内存不足、镜像版本不兼容。\n</context>\n\n<question>\nHow do I know if Milvus has started successfully?\n</question>\n\n# 任务要求：\n# 1. 首先，用英文回答<question>中的问题，回答必须严格基于<context>内的信息，禁止编造。\n# 2. 如果<context>信息不足以回答问题，请直接输出："Based on the provided information, I can\'t answer this question."\n# 3. 然后，在<translated></translated>标签内，提供上述英文回答的精准中文翻译。\n# 4. 输出格式示例：\n# [英文回答]\n# <translated>\n# [中文翻译]\n# </translated>\n'

- 使用 DeepSeek 提供的 deepseek-chat 模型根据提示生成响应。

In [29]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

To determine if Milvus has started successfully: 1. Check that the container status is "Up" using `docker ps`; 2. Verify the logs for the message 'Milvus started successfully'; 3. Ensure the client connects without errors.
<translated>
要判断 Milvus 是否启动成功：1. 使用 `docker ps` 查看容器状态是否为 "Up"；2. 检查日志中是否有 'Milvus started successfully' 消息；3. 确保客户端连接无错误。
</translated>
